#라이브러리 설치 및 불러오기

selenium
- driver: 크롬 브라우저 객체인듯 여러 내장 함수를 이용해서 코드 안에서 동적으로 페이지 조작 가능 ex)클릭

In [ ]:
!pip install selenium #셀리니움 설치
!apt-get update
!apt install chromium-chromedriver #chrome 드라이버 설치

     |████████████████████████████████| 979 kB 8.1 MB/s 
     |████████████████████████████████| 359 kB 54.3 MB/s 
     |████████████████████████████████| 138 kB 52.6 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 4.0 MB 41.2 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https:

In [ ]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


#크롤링

In [ ]:
#모든 영화를 크롤링 하기 위해선 페이지를 이동하는게 아닌 같은 페이지에 동적으로 로드해야됨
#받은 인자만큼 더보기 버튼 누르기
def getPage(pageRange):
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
  url = "https://serieson.naver.com/v2/movie/products?sortType=RATING"
  driver.implicitly_wait(1) #해줘야 전체 페이지 렌더링이 끝날때까지 기다림
  driver.get(url)
  for _ in range(pageRange):
    try:
      moreButton = driver.find_element_by_css_selector('#wrap > div.content_list > div > div.component_wrap > div.content_list_cont > a')
      moreButton.click()
      time.sleep(2) #새로운 영화 리스트와 더보기 버튼이 로드 될때까지 기다림
      # 2로 설정해야 정상적으로 로드되고 클릭
    except:
      break
    
  body = driver.find_element_by_tag_name('body')
  return body


In [ ]:
#로드를 끝낸 페이지에서 영화 정보를 담고 있는 태그를 크롤링
def getMovieList(pageRange):
  body = getPage(pageRange) #5개하면 15초 10개하면 25초 = 처음 렌더링 5초 + 1개에 2초

  movieUl = body.find_element_by_class_name("content_list_detail")
  movieList = movieUl.find_elements_by_class_name("item") #css 셀럭터로 하니까 이상한것도 뽑아옴

  return movieList

In [ ]:
movieList = getMovieList(3)
print( '영화 개수: {}'.format( len(movieList) ) ) #페이지 범위 하나 늘릴때마다 30개씩

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


영화 개수: 120


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  after removing the cwd from sys.path.


In [ ]:
for i in movieList:
  print("제목 :", i.find_element_by_class_name("title").text)
  print("평점 :", i.find_element_by_class_name("star_area").text)
  try:
    print("줄거리 :", i.find_element_by_class_name("description").text)
  except:
    print("줄거리 없음")
  
  print("\n")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  This is separate from the ipykernel package so we can avoid doing imports until


제목 : 뷰티플 마인드
평점 : 9.9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  """


줄거리 : 세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의 불협화음까지 닮았다! 혼자만의 쉼표, 도돌이표를 벗어나 난생 처음으로 함께하는 기적의 하모니를 꿈꿔보는데…


제목 : 뷰티플 마인드
평점 : 9.9
줄거리 : 세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의 불협화음까지 닮았다! 혼자만의 쉼표, 도돌이표를 벗어나 난생 처음으로 함께하는 기적의 하모니를 꿈꿔보는데…


제목 : 뷰티플 마인드(가치봄영화:시청각장애인용)
평점 : 9.9
줄거리 : 세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의 불협화음까지 닮았다! 혼자만의 쉼표, 도돌이표를 벗어나 난생 처음으로 함께하는 기적의 하모니를 꿈꿔보는데…


제목 : 뷰티플 마인드(가치봄영화:시청각장애인용)
평점 : 9.9
줄거리 : 세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의 불협화음까지 닮았다! 혼자만의 쉼표, 도돌이표를 벗어나 난생 처음으로 함께하는 기적의 하모니를 꿈꿔보는데…


제목 : 극장판 은혼 완결편 해결사여 영원하라
평점 : 9.9
줄거리 : "어떤 힘"에 의해 자신이 없는 미래의 세계에 날아가버린 긴토키는 붕괴해버린 에도의 거리를 두 눈으로 보게 된다. 멸망한 지구에 남은 것은 가난한 사람들, 이것을 기회로 정부와 싸우려는 양이지사, 법의 허점을 이용하는 불량배들, 그리고 꽁지 빠지게 도망치는 것이 성미에 맞지 않는 완고한 사람들뿐. 그런 에도에서 헤매이던 긴토키는 두 명의 젊은이와 만난다. 허리에 동야호목검을 차고 불량배를 쫓아내는 안경청년. 그리고, 거대한 개를 끌고 다니는 긴토키의 옷을 따라한 듯한 차이나 복장의 미녀. 그리고 마침내 긴토키는 계속해서 변해버린 미래의 모습과, 충격적인 사실과 마주하게 되는데...


제목 : 극장판 은혼 완결편 해결사여 영원하라
평점 : 9.9
줄거리 : "어떤 힘"에 의해 자신이 없는 미래의 세계에 날아가버린

#driver객체 -> DataFrame

크롬 페이지(driver 객체)에서 각 영화의 정보를 담고 있는 ul 태그를 순서대로 조회하여 정보를 추출 및 데이터 프레임에 저장


In [ ]:
import pandas as pd

def makeDF(movieList):
  df = pd.DataFrame(columns=['title', 'rate', 'description'])
  for i in movieList:
    try:
      description = i.find_element_by_class_name("description").text #줄거린 null값도 있음
    except:
      description = 'x'
    
    movie = {'title' : i.find_element_by_class_name("title").text,
            'rate' : i.find_element_by_class_name("star_area").text,
            'description' : description}
    df = df.append(movie,ignore_index=True)

  df = df.drop_duplicates(['title']) #중복되는 영화 제거
  noDescription = df[df['description'] == 'x'].index #줄거리 없는 영화 제거
  df = df.drop(noDescription)
  df = df.reset_index(drop=True) #인덱스 초기화

  return df

In [ ]:
df = makeDF(movieList)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  


,title,rate,description
0,뷰티플 마인드,9.9,"세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의..."
1,뷰티플 마인드(가치봄영화:시청각장애인용),9.9,"세상 극과 극의 개성으로 뭉친 ‘뷰티플마인드’의 멤버들 알고 보니, 마음 속 고민의..."
2,극장판 은혼 완결편 해결사여 영원하라,9.9,"""어떤 힘""에 의해 자신이 없는 미래의 세계에 날아가버린 긴토키는 붕괴해버린 에도의..."
3,몬스타엑스 : 더 드리밍,9.8,강렬한 퍼포먼스와 파워풀하고 완성도 높은 무대로 전 세계의 수많은 팬들에게 아름다운...
4,디지몬 어드벤처 2기: 우리들의 워 게임!,9.8,디지털 월드의 모험으로부터 반 년 후. 인터넷 상에는 전 세계의 디지털 데이터를 먹...
...,...,...,...
78,사운드 오브 뮤직,9.4,이곳의 견습 수녀인 마리아는 미사도 잊을 정도로 노래를 좋아하며 기도시간에 늦는 등...
79,월-E(패키지상품 : 더빙판 추가증정),9.4,"""니모를 찾아서""의 감독과 ""카""와 ""라따뚜이""를 탄생시킨 작가들이 다시 한 번 손..."
80,포레스트 검프,9.4,포레스트 검프(마이클 코너 험프레이스/톰 행크스 분)는 아이큐가 75이다. 그러나 ...
81,헬프,9.4,"1963년, 미국 남부 미시시피 잭슨 흑인 가정부는 백인 주인과 화장실도 같이 쓸 ..."


#DataFrame -> CSV

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.to_csv('movieList.csv')

#3000개

In [ ]:
movieList = getMovieList(300)
df = makeDF(movieList)
df.to_csv('movieList.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  import sys
/us

ProtocolError: ignored